In [4]:
import gym
from gym.spaces import Discrete
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [9]:
class OpenAIGymAPI:
    
    def __init__(self):
        self.env = 'CartPole-v1'
        self.observation_space = Discrete(2)
        self.action_space = self.env.action_space
        self.state = None
        self.total_reward = 0

    def initialize_training(self):
        self.state = self.env.reset()
        self.total_reward = 0
        return self.state
    
    def run_episodes(self, num_episodes=1000):
        rewards = []

        for episode in range(num_episodes):
            print(f"Running episode {episode + 1}...")
            episode_reward = self.run_episode()
            rewards.append(episode_reward)
        print(f"Finished running {num_episodes} episodes.")
        return rewards

    def run_episode(self):
        self.state = self.initialize_training()
        done = False
        self.total_reward = 0

        while not done:
            action = self.env.action_space.sample()
            next_state, reward, done, info = self.env.step(action)
            
            self.stimulate_neurons(reward)
            self.state = next_state

            self.total_reward += reward
        
        print(f"Episode finished with total reward: {self.total_reward}")
        return self.total_reward
    
    def stimulate_neurons(reward):
        if reward == 0:
            return "punish"        
        return "reward"
